Paquetes necesarios

In [2]:
import cv2  
import math 

from ultralytics import YOLO

## Usamos el modelo pre-entrenado de YOLO

In [ ]:
model = YOLO('yolo11n.pt')
filename = "TGC23_PdH_C0056cut.mp4"
vid = cv2.VideoCapture(filename)

while vid.isOpened():
    ret, img = vid.read()

    if ret:
        results = model(img, show=False)
        annotated_frame = results[0].plot()
        cv2.imshow("Detección de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Detección de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()